In [1]:
from ultralytics import YOLO
seg_model = YOLO("[model].pt")
results = seg_model("vid8/test/images")


image 1/90 d:\fiftyone\vid8\test\images\frame_000006.PNG: 384x640 2 cars, 199.9ms
image 2/90 d:\fiftyone\vid8\test\images\frame_000009.PNG: 384x640 2 cars, 156.7ms
image 3/90 d:\fiftyone\vid8\test\images\frame_000010.PNG: 384x640 2 cars, 185.9ms
image 4/90 d:\fiftyone\vid8\test\images\frame_000018.PNG: 384x640 2 cars, 182.4ms
image 5/90 d:\fiftyone\vid8\test\images\frame_000038.PNG: 384x640 2 cars, 237.9ms
image 6/90 d:\fiftyone\vid8\test\images\frame_000045.PNG: 384x640 2 cars, 227.9ms
image 7/90 d:\fiftyone\vid8\test\images\frame_000055.PNG: 384x640 3 cars, 160.1ms
image 8/90 d:\fiftyone\vid8\test\images\frame_000066.PNG: 384x640 2 cars, 185.5ms
image 9/90 d:\fiftyone\vid8\test\images\frame_000076.PNG: 384x640 3 cars, 184.0ms
image 10/90 d:\fiftyone\vid8\test\images\frame_000099.PNG: 384x640 2 cars, 174.8ms
image 11/90 d:\fiftyone\vid8\test\images\frame_000101.PNG: 384x640 1 car, 153.8ms
image 12/90 d:\fiftyone\vid8\test\images\frame_000104.PNG: 384x640 1 car, 178.8ms
image 13/90 d:

In [2]:
import fiftyone as fo
import numpy as np
import os


def read_yolo_detections_file(filepath):
    detections = []
    if not os.path.exists(filepath):
        return np.array([])

    with open(filepath) as f:
        lines = [line.rstrip('\n').split(' ') for line in f]

    for line in lines:
        detection = [float(l) for l in line]
        detections.append(detection)
    return np.array(detections)


def _uncenter_boxes(boxes):
    '''convert from center coords to corner coords'''
    boxes[:, 0] -= boxes[:, 2]/2.
    boxes[:, 1] -= boxes[:, 3]/2.


def _get_class_labels(predicted_classes, class_list):
    labels = predicted_classes.cpu().numpy().astype(int)
    labels = [class_list[l] for l in labels]
    return labels


def get_prediction_filepath(filepath, result_prediction_path, run_number=1, ):
    run_num_string = ""
    if run_number != 1:
        run_num_string = str(run_number)
    filename = filepath.split("/")[-1].rsplit(".", 1)[0]
    return f"{result_prediction_path}/{filename}.txt"


def convert_yolo_segmentations_to_fiftyone(
    yolo_segmentations,
    class_list
):

    detections = []
    boxes = yolo_segmentations.boxes.xywhn
    if not boxes.shape or yolo_segmentations.masks is None:
        return fo.Detections(detections=detections)

    _uncenter_boxes(boxes)
    masks = yolo_segmentations.masks.data
    labels = _get_class_labels(yolo_segmentations.boxes.cls, class_list)
    confidences = yolo_segmentations.boxes.conf.cpu().numpy().astype(float)
    for label, box, mask, confidence in zip(labels, boxes, masks, confidences):
        # convert to absolute indices to index mask
        w, h = mask.shape
        tmp = np.copy(box)
        tmp[2] += tmp[0]
        tmp[3] += tmp[1]
        tmp[0] *= h
        tmp[2] *= h
        tmp[1] *= w
        tmp[3] *= w
        tmp = [int(b) for b in tmp]
        y0, x0, y1, x1 = tmp
        sub_mask = mask[x0:x1, y0:y1]

        detections.append(
            fo.Detection(
                label=label,
                bounding_box=list(box),
                mask=sub_mask.cpu().numpy().astype(bool),
                confidence=confidence
            )
        )

    return fo.Detections(detections=detections)


def add_yolo_detections(
    samples,
    prediction_field,
    prediction_filepath,
    class_list
):

    prediction_filepaths = samples.values(prediction_filepath)
    # yolo_detections = [read_yolo_detections_file(
    #     pf) for pf in prediction_filepaths]
    yolo_seg_results = results
    segmentations = [convert_yolo_segmentations_to_fiftyone(
        yd, class_list) for yd in yolo_seg_results]
    samples.set_values(prediction_field, segmentations)


# Define the paths to the dataset
dataset_dir = "vid8/test"
data_path = f"{dataset_dir}/images"
labels_path = f"{dataset_dir}/labels"
result_prediction_path = f"{dataset_dir}/prediction"
name_prediction = "model_version_1"
classes = ["car"]

# Load the dataset
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv4Dataset,
    data_path=data_path,
    labels_path=labels_path,
    label_type="polylines",
    classes=['car']
)

filepaths = dataset.values("filepath")
# # For each file path, replace \\ to /
filepaths = [fp.replace("\\", "/") for fp in filepaths]
prediction_filepaths = [get_prediction_filepath(
    fp, result_prediction_path=result_prediction_path) for fp in filepaths]
dataset.set_values(
    "yolov8n_det_filepath",
    prediction_filepaths
)

add_yolo_detections(
    dataset,
    name_prediction,
    "yolov8n_det_filepath",
    classes
)

 100% |███████████████████| 90/90 [1.3s elapsed, 0s remaining, 70.3 samples/s]         


In [3]:
import fiftyone as fo
import numpy as np
import cv2

def polyline_to_mask(polyline, height=1680, width=2992):
    mask = np.zeros((height, width), dtype=np.uint8)
    # Denormalize the points
    points = np.array(polyline.points) * [width, height]
    points = points.astype(np.int32)
    points = points.reshape((-1, 1, 2))
    cv2.fillPoly(mask, [points], 255)
    return mask

def convert_polylines_to_masks(sample, height=1680, width=2992):
    ground_truth = sample["ground_truth"]
    if not hasattr(ground_truth, "polylines"):
        print(f"Sample {sample.id} 'ground_truth' field has no 'polylines'")
        return
    
    new_detections = []
    for polyline in ground_truth.polylines:
        # Convert polyline to mask using default dimensions
        mask = polyline_to_mask(polyline, height, width)
        
        # Compute the bounding box for the polyline
        points = np.array(polyline.points).reshape(-1, 2) * [width, height]
        min_x = int(np.min(points[:, 0]))
        max_x = int(np.max(points[:, 0]))
        min_y = int(np.min(points[:, 1]))
        max_y = int(np.max(points[:, 1]))
        
        bbox = [
            min_x / width,
            min_y / height,
            (max_x - min_x) / width,
            (max_y - min_y) / height
        ]
        
        # Crop the mask to the bounding box
        cropped_mask = mask[min_y:max_y, min_x:max_x]
        # Convert the cropped mask to boolean
        boolean_mask = cropped_mask.astype(bool)
        
        # Create a detection with mask
        new_detection = fo.Detection(label=polyline.label, mask=boolean_mask, bounding_box=bbox)
        new_detections.append(new_detection)
    
    if new_detections:
        sample["predictions"] = fo.Detections(detections=new_detections)
    
    # Delete the ground_truth field after processing
    del sample["ground_truth"]
    sample.save()

# Iterate over each sample to convert polylines to masks
for sample in dataset:
    convert_polylines_to_masks(sample)

# Save the modified dataset
dataset.save()

In [4]:
detection_results = dataset.evaluate_detections(
    name_prediction,
    eval_key="eval",
    compute_mAP=True,
    gt_field="predictions",
    use_masks=True,
)

Evaluating detections...
 100% |███████████████████| 90/90 [3.8s elapsed, 0s remaining, 41.9 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 90/90 [2.8s elapsed, 0s remaining, 46.7 samples/s]      


In [5]:
mAP = detection_results.mAP()
print(f"mAP = {mAP}")

mAP = 0.47933907894669225


In [6]:
session = fo.launch_app(dataset)
session.wait()

Notebook sessions cannot wait
